# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

Simimilarly, It is an adapted project from Mr.Mccullum, and I primarily follow his idea, use yfiannce to create my own stuff.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import math
import yfinance as yf

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [4]:
tickers=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers=tickers.to_list()
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

In [5]:
apple= yf.Ticker('AAPL')
print(apple.info['regularMarketOpen'])
print(apple.info)

#肯定是没问题的，所以问题在于这个要在一个cell里！

227.2
{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '(408) 996-1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts

In [6]:
apple_info=yf.Ticker('AAPL').info
I_WANT=apple_info['52WeekChange']
print(I_WANT)

0.20379066


In [7]:
two_stocks=yf.Tickers('AAPL MSFT')
two_stocks.tickers['AAPL'].info['52WeekChange']
#different from the following
two_stocks.tickers['AAPL'].info.get('52WeekChange')
# difference is that when the value is missing, 1 will raise an error, 2 will return a None


0.20379066

In [8]:
#prepare chunk
chunk_size=150
first_result=[]
for i in range(0,len(tickers),chunk_size):
    chunk_tickers=tickers[i:i+chunk_size]
    chunk_tickers_str=' '.join(chunk_tickers)
    print(chunk_tickers_str)
    #这里chunk就好了
    #接下来开始一次性request 150个
    chunk_stocks=yf.Tickers(chunk_tickers_str)
    for ticker in chunk_tickers:
        single_stock_info=chunk_stocks.tickers[ticker].info
        price=single_stock_info.get('regularMarketOpen')
        price_change=single_stock_info.get('52WeekChange')
        first_result.append(
            {
                'Ticker': ticker,
                'Price': price,
                '1 Year Price Change':price_change,
                'Number of Share to Buy':'N/A'
            }
        )
    


MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK.B BBY TECH BIIB BLK BX BK BA BKNG BWA BSX BMY AVGO BR BRO BF.B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW CE COR CNC CNP CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG
DLTR D DPZ DOV DOW DHI DTE DUK DD EMN ETN EBAY ECL EIX EW EA ELV EMR ENPH ETR EOG EPAM EQT EFX EQIX EQR ERIE ESS EL EG EVRG ES EXC EXPE EXPD EXR XOM FFIV FDS FICO FAST FRT FDX FIS FITB FSLR FE FI FMC F FTNT FTV FOXA FOX BEN FCX GRMN IT GE GEHC GEV GEN GNRC GD GIS GM GPC GILD GPN GL GDDY GS HAL HIG HAS HCA DOC HSIC HSY HES HPE HLT HOLX HD HON HRL HST HWM HPQ HUBB HUM

In [12]:
second_result=pd.DataFrame(first_result)

third_result=second_result.dropna(subset=['Price'])
third_result=third_result.dropna(subset=['1 Year Price Change'])
final_result= third_result.reset_index(drop=True)
final_result

,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,MMM,149.460,0.912274,N/A
1,AOS,67.060,-0.160321,N/A
2,ABT,126.435,0.133616,N/A
3,ABBV,187.630,0.086452,N/A
4,ACN,381.510,0.050620,N/A
...,...,...,...,...
495,XYL,125.000,-0.004459,N/A
496,YUM,132.270,0.014347,N/A
497,ZBRA,384.140,0.562422,N/A
498,ZBH,108.150,-0.152019,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [13]:
final_result.sort_values('1 Year Price Change', ascending = False, inplace = True)
final_result=final_result[0:50]
final_result.reset_index(drop=True, inplace=True) # if there isn't drop=True, then the new frame will have an extra index
final_result

,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,VST,168.520,2.851234,N/A
1,PLTR,102.670,2.548305,N/A
2,TPL,1332.100,1.681960,N/A
3,GEV,360.600,1.546895,N/A
4,UAL,106.620,1.539486,N/A
5,AXON,657.140,1.479037,N/A
6,TRGP,200.140,1.317404,N/A
7,CEG,298.227,1.311773,N/A
8,RCL,263.000,1.184208,N/A
9,HWM,127.400,1.138771,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [14]:
# 建立随时输入的一个函数
def portfolio_input():
    global portfolio_size
    portfolio_size=input("How much money you have?")

    try:
        float(portfolio_size)
    except ValueError:
        print("Enter some real shit!")
        portfolio_size=input("How much money you have?")

#试一下
portfolio_input()
print(portfolio_size)

#就算每个买多少
position_size= float(portfolio_size)/len(final_result)
print(position_size)

for i in final_result.index:
    final_result.loc[i,'Number of Share to Buy']=float(position_size)/final_result.loc[i,'Price']

final_result




100000
2000.0


,Ticker,Price,1 Year Price Change,Number of Share to Buy
0,VST,168.520,2.851234,11.868028
1,PLTR,102.670,2.548305,19.479887
2,TPL,1332.100,1.681960,1.501389
3,GEV,360.600,1.546895,5.546312
4,UAL,106.620,1.539486,18.758207
5,AXON,657.140,1.479037,3.043491
6,TRGP,200.140,1.317404,9.993005
7,CEG,298.227,1.311773,6.706301
8,RCL,263.000,1.184208,7.604563
9,HWM,127.400,1.138771,15.698587


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

Indeed, we need a more sophisticated strategy to win the market

In [17]:
# 一个的摸索例子，建立函数
from datetime import datetime,timedelta
months=int(input("how many months?"))
print(datetime.today())
print(timedelta(days=30 * months))

end_date=datetime.today()
start_date=end_date-timedelta(days=30 * months)
print(start_date)
print(end_date)

#print(apple.history(start=start_date,end=end_date))
apple_history=apple.history(start=start_date,end=end_date)
old_price=apple_history['Close'].iloc[0]
new_price=apple_history['Close'].iloc[-1]
stock_return1=(new_price-old_price)/old_price
print(stock_return1)


2025-02-05 08:50:06.355033
360 days, 0:00:00
2024-02-11 08:50:06.355284
2025-02-05 08:50:06.355284
0.24842296067464414


In [16]:
#一个的例子，建立函数
from datetime import datetime,timedelta 
# 这里就构想，第二个parameter是一个object
def stock_return(months, one_stock):
    #时间
    end_date=datetime.today()
    start_date=end_date-timedelta(days=30 * months)
    #找价格
    stock_history=one_stock.history(start=start_date,end=end_date)
    global old_price
    global new_price
    old_price=float(stock_history['Close'].iloc[0])
    new_price=float(stock_history['Close'].iloc[-1])
    #Calculate return
    Stock_Return=(new_price-old_price)/old_price
    return float(Stock_Return)

print(stock_return(12,yf.Ticker('AAPL')))
print(old_price)
#牛逼！

0.24842296067464414
186.4752655029297


In [26]:
#建立函数,复制上面的
from datetime import datetime,timedelta 
def stock_return(months, one_stock):
    end_date=datetime.today()
    start_date=end_date-timedelta(days=30 * months)
    stock_history=one_stock.history(start=start_date,end=end_date)
    global old_price
    global new_price
    old_price=float(stock_history['Close'].iloc[0])
    new_price=float(stock_history['Close'].iloc[-1])
    Stock_Return=(new_price-old_price)/old_price
    return float(Stock_Return)

#prepare chunk
chunk_size=150
first_result=[]
for i in range(0,len(tickers),chunk_size):
    chunk_tickers=tickers[i:i+chunk_size]
    chunk_tickers_str=' '.join(chunk_tickers)
    print(chunk_tickers_str)
    #这里chunk就好了
    #接下来开始一次性request 150个
    chunk_stocks=yf.Tickers(chunk_tickers_str)
    for ticker in chunk_tickers:
        try:
            single_stock=chunk_stocks.tickers[ticker]
            price=new_price
            price_change_1_month=stock_return(1,single_stock)
            price_change_3_month=stock_return(3,single_stock)
            price_change_6_month=stock_return(6,single_stock)
            price_change_12_month=stock_return(12,single_stock)
            
            first_result.append(
                {
                    'Ticker': ticker,
                    'Price': price,
                    '1 Month Price Change':price_change_1_month,
                    '1 Month Return Percentile':"N/A",
                    '3 Months Price Change':price_change_3_month,
                    '3 Months Return Percentile':"N/A",
                    '6 Months Price Change':price_change_6_month,
                    '6 Months Return Percentile':"N/A",
                    '12 Months Price Change':price_change_12_month,
                    '12 Months Return Percentile':"N/A",
                    'HQM(high-quality momentum score)':'N/A',
                    'Number of Share to Buy':'N/A'
                }
            )
        except Exception as e:
            print(f"{ticker} is wrong.")
            continue

# 2024-01-09跑成
        


MMM AOS ABT ABBV ACN ADBE AMD AES AFL A APD ABNB AKAM ALB ARE ALGN ALLE LNT ALL GOOGL GOOG MO AMZN AMCR AEE AEP AXP AIG AMT AWK AMP AME AMGN APH ADI ANSS AON APA APO AAPL AMAT APTV ACGL ADM ANET AJG AIZ T ATO ADSK ADP AZO AVB AVY AXON BKR BALL BAC BAX BDX BRK.B BBY TECH BIIB BLK BX BK BA BKNG BWA BSX BMY AVGO BR BRO BF.B BLDR BG BXP CHRW CDNS CZR CPT CPB COF CAH KMX CCL CARR CAT CBOE CBRE CDW CE COR CNC CNP CF CRL SCHW CHTR CVX CMG CB CHD CI CINF CTAS CSCO C CFG CLX CME CMS KO CTSH CL CMCSA CAG COP ED STZ CEG COO CPRT GLW CPAY CTVA CSGP COST CTRA CRWD CCI CSX CMI CVS DHR DRI DVA DAY DECK DE DELL DAL DVN DXCM FANG DLR DFS DG


KeyboardInterrupt: 

In [19]:

print(len(first_result))
second_result=pd.DataFrame(first_result)
columns_list=second_result.columns.tolist()
third_result=second_result.dropna(subset=columns_list)
final_result= third_result.reset_index(drop=True)
print(len(final_result))
final_result


379
379


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,186.475266,0.164172,N/A,0.145913,N/A,0.234668,N/A,0.983487,N/A,N/A,N/A
1,AOS,76.471390,-0.042187,N/A,-0.132009,N/A,-0.174159,N/A,-0.178195,N/A,N/A,N/A
2,ABT,79.897285,0.148067,N/A,0.125564,N/A,0.199422,N/A,0.170630,N/A,N/A,N/A
3,ABBV,110.282539,0.064638,N/A,-0.043732,N/A,0.017598,N/A,0.137513,N/A,N/A,N/A
4,ACN,166.987061,0.114679,N/A,0.085692,N/A,0.245018,N/A,0.079029,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
374,POOL,143.836182,0.020571,N/A,-0.082913,N/A,-0.021540,N/A,-0.125195,N/A,N/A,N/A
375,PPG,387.914917,-0.033023,N/A,-0.111971,N/A,-0.056394,N/A,-0.193801,N/A,N/A,N/A
376,PPL,138.018051,0.059250,N/A,0.047422,N/A,0.118781,N/A,0.322906,N/A,N/A,N/A
377,PFG,25.406181,0.036880,N/A,-0.053813,N/A,0.097895,N/A,0.058248,N/A,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [20]:
Time_period_list=['1 Month','3 Months','6 Months','12 Months']
for i in Time_period_list:
    final_result[f"{i} Return Percentile"]=final_result[f"{i} Price Change"].rank(pct=True)

final_result

        
        


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,186.475266,0.164172,0.984169,0.145913,0.923483,0.234668,0.786280,0.983487,0.984169,N/A,N/A
1,AOS,76.471390,-0.042187,0.171504,-0.132009,0.137203,-0.174159,0.089710,-0.178195,0.121372,N/A,N/A
2,ABT,79.897285,0.148067,0.965699,0.125564,0.891821,0.199422,0.736148,0.170630,0.569921,N/A,N/A
3,ABBV,110.282539,0.064638,0.717678,-0.043732,0.406332,0.017598,0.398417,0.137513,0.514512,N/A,N/A
4,ACN,166.987061,0.114679,0.918206,0.085692,0.825858,0.245018,0.799472,0.079029,0.414248,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
374,POOL,143.836182,0.020571,0.435356,-0.082913,0.269129,-0.021540,0.316623,-0.125195,0.189974,N/A,N/A
375,PPG,387.914917,-0.033023,0.197889,-0.111971,0.182058,-0.056394,0.263852,-0.193801,0.105541,N/A,N/A
376,PPL,138.018051,0.059250,0.686016,0.047422,0.741425,0.118781,0.627968,0.322906,0.757256,N/A,N/A
377,PFG,25.406181,0.036880,0.546174,-0.053813,0.372032,0.097895,0.577836,0.058248,0.393140,N/A,N/A


In [21]:
final_result.iloc[0,[3,5,7,9]]

1 Month Return Percentile      0.984169
3 Months Return Percentile     0.923483
6 Months Return Percentile      0.78628
12 Months Return Percentile    0.984169
Name: 0, dtype: object

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [24]:

for i in final_result.index:
    final_result.loc[[i],['HQM(high-quality momentum score)']]=final_result.iloc[i,[3,5,7,9]].mean()

final_result

,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,MMM,186.475266,0.164172,0.984169,0.145913,0.923483,0.234668,0.786280,0.983487,0.984169,0.919525,N/A
1,AOS,76.471390,-0.042187,0.171504,-0.132009,0.137203,-0.174159,0.089710,-0.178195,0.121372,0.129947,N/A
2,ABT,79.897285,0.148067,0.965699,0.125564,0.891821,0.199422,0.736148,0.170630,0.569921,0.790897,N/A
3,ABBV,110.282539,0.064638,0.717678,-0.043732,0.406332,0.017598,0.398417,0.137513,0.514512,0.509235,N/A
4,ACN,166.987061,0.114679,0.918206,0.085692,0.825858,0.245018,0.799472,0.079029,0.414248,0.739446,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
374,POOL,143.836182,0.020571,0.435356,-0.082913,0.269129,-0.021540,0.316623,-0.125195,0.189974,0.30277,N/A
375,PPG,387.914917,-0.033023,0.197889,-0.111971,0.182058,-0.056394,0.263852,-0.193801,0.105541,0.187335,N/A
376,PPL,138.018051,0.059250,0.686016,0.047422,0.741425,0.118781,0.627968,0.322906,0.757256,0.703166,N/A
377,PFG,25.406181,0.036880,0.546174,-0.053813,0.372032,0.097895,0.577836,0.058248,0.393140,0.472296,N/A


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [25]:
eventual_result=final_result.sort_values(by='HQM(high-quality momentum score)',ascending=False)
Best_Momentum_stocks=eventual_result.iloc[0:50].reset_index(drop=True)
Best_Momentum_stocks

,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,PLTR,164.048767,0.367624,1.000000,0.858089,1.000000,2.459847,1.000000,3.144910,1.000000,1.0,N/A
1,AXON,203.539871,0.161553,0.981530,0.414208,0.997361,0.816192,0.992084,1.485509,0.994723,0.991425,N/A
2,CEG,244.029663,0.159112,0.978892,0.269964,0.989446,0.618908,0.976253,1.400256,0.992084,0.984169,N/A
3,NFLX,89.091766,0.128239,0.947230,0.248989,0.984169,0.569344,0.965699,0.783401,0.965699,0.965699,N/A
4,FFIV,99.766815,0.154332,0.976253,0.239375,0.976253,0.561354,0.960422,0.610335,0.928760,0.960422,N/A
5,LYV,414.553802,0.145355,0.960422,0.188282,0.955145,0.606205,0.973615,0.654302,0.941953,0.957784,N/A
6,DAL,85.339890,0.150233,0.970976,0.142810,0.915567,0.762438,0.986807,0.720142,0.955145,0.957124,N/A
7,CCL,76.949997,0.122669,0.939314,0.144003,0.920844,0.834123,0.994723,0.748870,0.960422,0.953826,N/A
8,FOXA,82.796951,0.100947,0.886544,0.223192,0.968338,0.398473,0.931398,0.823220,0.973615,0.939974,N/A
9,IBM,269.979858,0.187677,0.994723,0.247341,0.981530,0.392239,0.928760,0.458662,0.854881,0.939974,N/A


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [219]:
# 建立随时输入的一个函数
def portfolio_input():
    global portfolio_size
    portfolio_size=input("How much money you have?")

    try:
        float(portfolio_size)
    except ValueError:
        print("Enter some real shit!")
        portfolio_size=input("How much money you have?")

#试一下
portfolio_input()
print(portfolio_size)

#就算每个买多少
position_size= float(portfolio_size)/len(Best_Momentum_stocks)
print(position_size)

for i in Best_Momentum_stocks.index:
    Best_Momentum_stocks.loc[i,'Number of Share to Buy']=math.floor(float(position_size)/Best_Momentum_stocks.loc[i,'Price'])

Best_Momentum_stocks

100000
2000.0


,Ticker,Price,1 Month Price Change,1 Month Return Percentile,3 Months Price Change,3 Months Return Percentile,6 Months Price Change,6 Months Return Percentile,12 Months Price Change,12 Months Return Percentile,HQM(high-quality momentum score),Number of Share to Buy
0,GEV,74.891090,0.120240,0.991404,0.374182,0.997135,1.066927,1.000000,1.582422,1.000000,0.997135,26
1,AVGO,47.620228,0.338248,1.000000,0.266941,0.982808,0.345704,0.951289,1.084367,0.985673,0.979943,41
2,FOX,30.274151,0.061006,0.962751,0.226579,0.977077,0.399204,0.971347,0.666333,0.945559,0.964183,66
3,DECK,65.489998,0.026717,0.899713,0.286946,0.988539,0.384113,0.965616,0.725069,0.962751,0.954155,30
4,FOXA,71.075516,0.056071,0.957020,0.183136,0.951289,0.369669,0.959885,0.617552,0.934097,0.950573,28
5,KMI,19.357944,0.052064,0.948424,0.158779,0.919771,0.438099,0.977077,0.680618,0.951289,0.94914,103
6,ETR,108.750000,0.032832,0.908309,0.182195,0.948424,0.459715,0.985673,0.548516,0.908309,0.937679,18
7,EQT,300.170013,0.138211,0.994269,0.315346,0.991404,0.342947,0.945559,0.374116,0.799427,0.932665,6
8,ANET,66.693390,0.102405,0.982808,0.108196,0.859599,0.281949,0.893983,0.824370,0.974212,0.92765,29
9,NRG,17.250000,0.051318,0.945559,0.102917,0.851003,0.320161,0.916905,0.979596,0.982808,0.924069,115


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [220]:
# put it as excel
Best_Momentum_stocks.to_excel('My Second Shit.xlsx',
                         sheet_name="1",
                         index=False)

## Saving Our Excel Output

As before, saving our Excel output is very easy: